# First CNN model

In [58]:
import pandas as pd
from pathlib import Path
import numpy as np
from PIL import Image
import os
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.callbacks import EarlyStopping

In [13]:
def preprocess_images(X: pd.DataFrame, resize_size: tuple)-> np.ndarray:
    """
    INPUT
    X: the DataFrame from which we want to retrieve the image names in the 'Image_name' column

    resize_size: a tuple to which dimensions the images should be resized
    For example (100, 100) resizes the images to 100 x 100 pixels

    OUTPUT
    A numpy array of shape (number of images, pixel length, pixel height, color channels)
    """

    output = []

    for image_name in X['Image_name']:
        image_path = os.path.join("../raw_data/Images/", image_name)
        image = Image.open(image_path)
        image = image.resize(resize_size) # Resizing for speed purposes
        image_array = np.array(image)
        output.append(image_array)

    return np.array(output)

In [14]:
def get_data(cache_path: Path) -> pd.DataFrame:
    """Retrieve data from `cache_path`"""
    df = pd.read_csv(cache_path, delimiter=" ", index_col="Id")
    return df

In [16]:
X_train = get_data("../raw_data/train_x.csv")
y_train = get_data("../raw_data/train_y.csv")

X_test = get_data("../raw_data/test_x.csv")
y_test = get_data("../raw_data/test_y.csv")

current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
images_folder = os.path.join(parent_directory, 'raw_data', 'Images')
images_filenames = os.listdir(images_folder)

In [10]:
print((X_train.shape, y_train.shape), (X_test.shape, y_test.shape))

((51300, 2), (51300, 2)) ((5700, 2), (5700, 2))


In [19]:
def preprocess_images(X: pd.DataFrame, resize_size: tuple)-> np.ndarray:
    """
    INPUT
    X: the DataFrame from which we want to retrieve the image names in the 'Image_name' column

    resize_size: a tuple to which dimensions the images should be resized
    For example (100, 100) resizes the images to 100 x 100 pixels

    OUTPUT
    A numpy array of shape (number of images, pixel length, pixel height, color channels)
    """

    output = []

    for image_name in X['Image_name']:
        image_path = os.path.join("../raw_data/Images/", image_name)
        image = Image.open(image_path)
        image = image.resize(resize_size) # Resizing for speed purposes
        image_array = np.array(image)
        output.append(image_array)

    return np.array(output)

images_train, images_test = get_data("../raw_data/Images/")


In [31]:
X_train = preprocess_images(X_train, (100,100))

In [39]:
X_test = preprocess_images(X_test, (100,100))

In [32]:
X_train.shape

(51300, 100, 100, 3)

In [33]:
X_train

array([[[[202, 196, 172],
         [200, 194, 170],
         [195, 189, 165],
         ...,
         [196, 190, 168],
         [196, 190, 168],
         [196, 190, 168]],

        [[205, 199, 175],
         [201, 195, 171],
         [199, 193, 169],
         ...,
         [195, 189, 167],
         [195, 189, 167],
         [196, 190, 168]],

        [[203, 197, 173],
         [203, 197, 173],
         [203, 197, 173],
         ...,
         [193, 187, 165],
         [195, 189, 167],
         [196, 190, 168]],

        ...,

        [[ 91,  96,  73],
         [ 87,  92,  69],
         [ 88,  93,  70],
         ...,
         [ 96, 101,  78],
         [ 96, 101,  78],
         [ 96, 101,  78]],

        [[ 92,  97,  74],
         [ 87,  92,  69],
         [ 91,  96,  73],
         ...,
         [ 95, 100,  77],
         [ 95, 100,  77],
         [ 95, 100,  77]],

        [[ 90,  95,  72],
         [ 89,  94,  71],
         [ 92,  97,  74],
         ...,
         [ 94,  99,  76],
        

In [34]:
y_train = y_train['Genre_id']

KeyError: 'Genre_id'

In [35]:
y_train

Id
0         1
1         4
2        10
3         9
4        26
         ..
51295    17
51296    17
51297    17
51298    17
51299    17
Name: Genre_id, Length: 51300, dtype: int64

In [46]:
y_test = y_test['Genre_id']

In [36]:
(X_train.shape, y_train.shape)

((51300, 100, 100, 3), (51300,))

## Normalise train and test data - **add this to preprocessing**

In [40]:
X_train = X_train / 255
X_test = X_test / 255

In [41]:
print(X_train.shape)
print(X_test.shape)

(51300, 100, 100, 3)
(5700, 100, 100, 3)


## One-Hot-Encode target y - **add this to preprocessing**

In [47]:
y_test 

Id
0       16
1       28
2        1
3       21
4        0
        ..
5695    17
5696    17
5697    17
5698    17
5699    17
Name: Genre_id, Length: 5700, dtype: int64

In [48]:
len(np.unique(y_test))

30

In [49]:
y_train_cat = to_categorical(y_train, num_classes=30)
y_test_cat = to_categorical(y_test, num_classes=30)

In [52]:
y_train_cat.shape, y_test_cat.shape

((51300, 30), (5700, 30))

In [61]:
X_train.shape, y_train_cat.shape

((51300, 100, 100, 3), (51300, 30))

## CNN - dummy model

In [65]:
def initialize_model():

    model = models.Sequential()

    ### First Convolution & MaxPooling
    model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3), activation='relu', padding='same'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    
    ### Second Convolution & MaxPooling
    model.add(layers.Conv2D(16, (3,3), input_shape=(100, 100, 3), activation='relu', padding='same'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    
    ### Flattening
    model.add(layers.Flatten())
    
    ### One Fully Connected layer - "Fully Connected" is equivalent to saying "Dense"
    model.add(layers.Dense(30, activation='relu'))
    
    ### Last layer - Classification Layer with 10 outputs corresponding to 10 digits
    model.add(layers.Dense(30, activation='softmax'))
    
    ### Model compilation
    model.compile(loss='categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])
    
    return model

In [66]:
model = initialize_model()
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 100, 100, 8)       392       
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 50, 50, 8)         0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 50, 50, 16)        1168      
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 25, 25, 16)        0         
 g2D)                                                            
                                                                 
 flatten_4 (Flatten)         (None, 10000)             0         
                                                                 
 dense_8 (Dense)             (None, 10)               

In [67]:
model = initialize_model()
es = EarlyStopping(patience=10, restore_best_weights=True)

model.fit(X_train, y_train_cat,
         validation_split = 0.2,
         epochs=5, 
         batch_size=250,
         verbose=1, 
         callbacks=[es])